In [ ]:
import pandas as pd
import requests
import datetime as dt
import yfinance as yf

hello


In [3]:
def get_forex_data(base_currency: str, quote_currency: str, start_date: str, end_date: str):


    """
    Fetch historical Forex data for a currency pair from Yahoo Finance.

    Args:
        base_currency (str): The base currency code, e.g., 'USD'
        quote_currency (str): The quote currency code, e.g., 'JPY'
        period (str): Data period (default '1y'). Options like '1d', '5d', '1mo', etc.

    """

    if quote_currency:
        ticker = f"{base_currency.upper()}{quote_currency.upper()}=X"
    else:
        ticker = base_currency  # full ticker like "^DXY"

    fx = yf.Ticker(ticker)

    data = fx.history(start = start_date, end = end_date, period = "1d")
    data.index = pd.to_datetime(data.index)
    data["time"] = pd.to_datetime(data.index).to_series().dt.strftime("%Y-%m")
    data.index = data.index.tz_localize(None)
    data = data.rename(columns={'Close': f'Close_{quote_currency}'})
    data = data.drop(["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"], axis = 1)
    

    return data

In [4]:
def merged_data(base_currency, quote_currency_list, start_date, end_date):

  merged_df = None

  for i in quote_currency_list:
    df = get_forex_data(base_currency, i , start_date, end_date)

    if merged_df is None:
      merged_df = df
    else:
      df = df.drop('time', axis = 1)
      merged_df = pd.merge(merged_df, df, left_index = True, right_index = True, how = "left")

  return merged_df

merged_data("USD", ["EUR", "CAD", "AUD", "JPY", "GBP"], "2005-01-01", "2013-05-01")


,Close_EUR,time,Close_CAD,Close_AUD,Close_JPY,Close_GBP
Date,,,,,,
2005-01-03,0.74239,2005-01,1.20780,NaN,102.739998,0.52504
2005-01-04,0.75290,2005-01,1.22120,NaN,104.339996,0.53090
2005-01-05,0.75301,2005-01,1.22440,NaN,103.930000,0.53036
2005-01-06,0.75855,2005-01,1.23530,NaN,104.889999,0.53291
2005-01-07,0.76564,2005-01,1.23130,NaN,104.889999,0.53439
...,...,...,...,...,...,...
2013-04-24,0.76956,2013-04,1.02703,0.97456,99.677002,0.65660
2013-04-25,0.76820,2013-04,1.02526,0.97286,99.419998,0.65446
2013-04-26,0.76880,2013-04,1.02015,0.97145,99.342003,0.64780


In [5]:
import streamlit as st
import pandas as pd
import numpy as np

# ---------- PAGE CONFIG ----------
st.set_page_config(page_title="USD Strength Dashboard", layout="wide")

# ---------- CUSTOM CSS ----------
st.markdown("""
    <style>
    [data-testid="stMetric"] {
        background-color: #1c1c1c;
        color: white;
        border-radius: 10px;
        padding: 1rem;
        margin-bottom: 1rem;
        box-shadow: 0px 0px 8px rgba(255, 255, 255, 0.05);
        text-align: center;
    }
    .big-font {
        font-size: 30px !important;
        font-weight: 600;
    }
    </style>
""", unsafe_allow_html=True)

# ---------- SIDEBAR ----------
st.sidebar.title("⚙️ Model Selection & Inputs")
model_choice = st.sidebar.radio("Select Analysis Method:", ["Logistic Regression", "Scorecard", "Gradient Boosting"])

# -------- SCORECARD OPTIONS --------
if model_choice == "Scorecard":
    st.sidebar.subheader("📊 Scorecard Weights")
    weights = {
        "Interest Rate Diff": st.sidebar.slider("Interest Rate Differential", 0.0, 1.0, 0.3),
        "Inflation Diff": st.sidebar.slider("Inflation Differential", 0.0, 1.0, 0.3),
        "Trade Balance": st.sidebar.slider("Trade Balance", 0.0, 1.0, 0.2),
        "Market Sentiment": st.sidebar.slider("Market Risk Sentiment", 0.0, 1.0, 0.2)
    }

# ---------- HEADER ----------
st.title("💵 USD Strength Diagnostic Dashboard")
st.markdown("Use different models to analyze the strength of the US dollar and determine key drivers behind its movement.")

# ---------- KPI METRICS ----------
col1, col2, col3 = st.columns(3)
col1.metric("USD Strength", "72", "+3")
col2.metric("3M Change", "-1.4%", "-")
col3.metric("Top Driver", "Interest Rate Differential", "")

# ---------- MAIN ANALYSIS ----------
st.subheader(f"📈 Results from: {model_choice}")

# --- DYNAMIC DISPLAY ---
if model_choice == "Logistic Regression":
    st.write("ℹ️ Logistic Regression results will appear here.")
    # TODO: Insert model output table / probability
    # st.table(predictions_df)

elif model_choice == "Scorecard":
    st.write("📋 Custom Scorecard Analysis")
    # Dummy example:
    data = {
        "Variable": list(weights.keys()),
        "Weight": list(weights.values()),
        "Value": [0.8, -0.4, 0.6, -0.1],  # Replace with actual values
    }
    df = pd.DataFrame(data)
    df["Weighted Score"] = df["Weight"] * df["Value"]
    df["Impact"] = df["Weighted Score"].apply(lambda x: "↑" if x > 0 else "↓")

    st.dataframe(df.style.background_gradient(cmap="RdYlGn"))

    st.metric("Scorecard USD Score", round(df["Weighted Score"].sum(), 2))

elif model_choice == "Gradient Boosting":
    st.write("🚀 Gradient Boosting Model Output")
    # TODO: Show feature importances / predictions
    # st.plotly_chart(shap_plot)

2025-06-20 14:49:26.024 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-20 14:49:26.036 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-20 14:49:29.538 
  command:

    streamlit run d:\Raghav\Georgetown\Summer Clinic\venv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-06-20 14:49:29.541 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-20 14:49:29.543 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-20 14:49:29.546 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-20 14:49:29.547 Thread 'MainThread': missing ScriptRunContext! This warning can be ig